In [7]:
%pip install fuzzywuzzy
%pip install Levenshtein
%pip install yahoo-finance
%pip install yfinance
%pip install -U nbformat
%pip install scipy

In [8]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import yfinance as yf
import plotly.express as px
from functools import lru_cache

In [64]:
df1 = pd.read_csv('/content/stock_part1.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ID           200000 non-null  int64 
 1   Date         200000 non-null  object
 2   Ticker       200000 non-null  object
 3   Open         200000 non-null  object
 4   High         200000 non-null  object
 5   Low          200000 non-null  object
 6   Close        200000 non-null  object
 7   Adj Close    200000 non-null  object
 8   Volume       200000 non-null  object
 9   Dividend     200000 non-null  object
 10  Split Ratio  161520 non-null  object
dtypes: int64(1), object(10)
memory usage: 16.8+ MB


In [33]:
df1

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio
0,1,2023-09-03,META,"258,44","909,63","1380,05","595,86","61,54","8,846,040,350","3,57",1:1
1,2,2024-08-28,NVDA,"678,96","725,03","185,51","295,83","843,78",7346935699,"2,85",3:2
2,3,2022-05-10,INTU,"1348,94","215,00","831,86","874,05","336,19","8,027,589,419","2,28",2:1
3,4,2017-12-26,NVDA,"186,39","1111,85","508,65","511,49","952,63",1939.5M,"4,51",3:2
4,5,2022-01-20,WORK,"1425,94","692,20","340,75","378,58","1493,44",8984007687,"1,93",NaN
...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,2019-07-27,TSLA,"269,17","144,45","418,85","588,19","893,89",4184.54M,"4,71",3:2
199996,199997,2021-09-13,VMW,"315,78","262,78","89,13","172,09","659,42","5,782,458,138","1,04",3:2
199997,199998,2016-08-19,AAPL,"327,66","897,77","1402,05","140,90","904,11",3104296586,"3,21",NaN
199998,199999,2020-06-24,DOCU,"141,99","739,41","285,76","529,92","1201,64",45454067,"1,51",NaN


In [32]:
df1['Date_original'] = df1['Date']

df1['Date'] = pd.to_datetime(df1['Date'], format="mixed", errors='coerce')
invalid_dates = df1[df1['Date'].isna()]
invalid_values = df1.loc[invalid_dates.index, 'Date_original']
df1 = df1.drop(columns=["Date_original"])

nerr = len(invalid_dates) # quantas datas invalidas tem, ex: 12/63/2000 ou 12/81/2023
percent = nerr / len(df1["Date"]) * 100
print(f"Percentual de erros: {percent}%")
print(f"Número de erros: {nerr}")

Percentual de erros: 1.015%
Número de erros: 2030


In [12]:
from datetime import datetime

future_dates = df1[df1['Date'] > datetime.now()]["Date"]
print(future_dates)

715      2025-11-02
948      2025-09-05
1036     2025-07-03
2830     2025-09-05
2880     2025-10-01
            ...    
197447   2025-09-03
197677   2025-10-03
198021   2025-09-05
198338   2025-09-05
199004   2025-08-04
Name: Date, Length: 498, dtype: datetime64[ns]


In [13]:
maiormes = 0
for i in future_dates.dt.strftime('%Y-%m-%d'):
    mesatual = int(i[-2:])
    maiormes = ( mesatual if mesatual > maiormes else maiormes)
print(maiormes)

5


In [55]:
future_dates = future_dates.apply(lambda x: x.replace(day=x.month, month=x.day) if pd.notnull(x) else x)

df1.loc[future_dates.index, "Date"] = future_dates
df1["Date"] = df1["Date"].apply(lambda x: x.replace(tzinfo=None))

In [15]:
ticker_counts = df1['Ticker'].value_counts()
print(ticker_counts.describe())

count    3307.000000
mean       60.477774
std       551.331473
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max      6161.000000
Name: count, dtype: float64


In [16]:
#Acho melhor baixar um df com os tickers válidos, para não precisar fazer requisição na api do yahoo
df1["Ticker"] = df1["Ticker"].str.upper().str.strip()



ticker_counts = df1["Ticker"].value_counts()


valid_tickers = list(set(
    ticker_counts[ticker_counts > 10].index.tolist()
))
valid_tickers = [t.upper() for t in valid_tickers]
tickers = pd.read_csv("/content/tickers_new.csv")

@lru_cache(maxsize=None)
def is_valid_ticker(ticker):
    if ticker in tickers["Symbol"].values:

        return True
    return False
c= 0
def get_fuzzy_match(ticker, candidates, min_score=65):
    if not ticker or len(ticker) < 2:
        return None
    candidates_filtered = [t for t in candidates if abs(len(t) - len(ticker)) <= 1]
    if not candidates_filtered:
        return None
    match, score = process.extractOne(ticker, candidates_filtered)
    if score >= min_score:
        return match
    return None


corrections = {}
for ticker in ticker_counts[ticker_counts <= 10].index:
    if not is_valid_ticker(ticker):
        corrected = get_fuzzy_match(ticker, valid_tickers)
        if corrected:
            corrections[ticker] = corrected
    else:
        c+=1
        #print(ticker)

print("Correções feitas:")
for k, v in corrections.items():
    print(f"{k} → {v}")


df1["Ticker"] = df1["Ticker"].replace(corrections)

Correções feitas:
RBM → CRM
BYPL → PYPL
SAOT → SPOT
BMZN → AMZN
INTB → INTF
AMZB → AMZN
DOCG → DOCU
IDM → IBM
IAPL → AAPL
ADFE → ADBE
AFZN → AMZN
AMW → VMW
UBJR → UBER
AMZ → AMZN
CSRO → CSCO
IXTU → INTU
CRN → CRM
ZMW → ZM
DMD → EMD
AAPH → AAPL
IXM → IBM
GTOG → GOOG
MSRT → MSFT
XAPL → AAPL
WBM → IBM
SHHP → SHOP
OUCL → ORCL
UBEF → UBER
DPCU → DOCU
PINL → PINS
NLLX → NFLX
AQPL → AAPL
WWTR → TWTR
CSXO → CSCO
AHD → AMD
UKER → UBER
CQM → CRM
UBEP → UBER
KMW → VMW
INLC → INTC
BABT → BABA
UBEQ → UBER
IBY → IBM
VLW → VMW
BABS → BABA
BYBA → BABA
MAPL → AAPL
IZTU → INTU
BHBA → BABA
AMK → AMD
ADBM → ADBE
ZRM → ZM
SPOH → SPOT
XYPL → PYPL
YWTR → TWTR
YNAP → SNAP
CRQ → CRM
AMJ → AMD
NVJA → NVDA
NFLJ → NFLX
BABJ → BABA
JBM → IBM
PBPL → PYPL
MNTU → INTU
AMQ → AMD
GBER → UBER
SHLP → SHOP
NFHX → NFLX
CRX → CRM
SHUP → SHOP
IMD → EMD
BAZA → BABA
TWTN → TWTR
PBNS → PINS
MSUT → MSFT
GORK → WORK
BABI → BABA
TETA → META
BABQ → BABA
WSCO → CSCO
MOTA → META
TABA → BABA
UBM → IBM
MEDA → META
CHM → CRM
VMN → VMW
P

In [17]:
print(c)

164


In [35]:
numerics_columns = ["Open", "High", "Low", "Close", "Adj Close", "Volume", "Dividend"]
def get_non_numeric_characters(column):
  print(column.name)
  tot = set("")
  for index, value in column.items():
    tot = tot | set([c for c in value if not c.isdigit()])
  print(tot)

def count_comma(column):
    count = 0
    for index,value in column.items():
        if isinstance(value, str) and len(value) >= 3:
            if value[-3] in [',', '.']:
                count += 1
    print(count)



for i in numerics_columns:
  get_non_numeric_characters(df1[i])
  count_comma(df1[i])



Open
{','}
200000
High
{','}
200000
Low
{','}
200000
Close
{','}
200000
Adj Close
{','}
200000
Volume
{',', 'i', 'b', 'n', 'B', 'K', '.', 'o', ' ', 'M', 'l'}
3503
Dividend
{','}
200000


In [36]:
equivalents_numeric_columns = numerics_columns
equivalents_numeric_columns.remove("Volume")

def change_caracter (value):
  value = value.strip()
  value = value.strip("$")
  value = value.replace("Z", "2")
  value = value.replace(".", ",")
  return value

for i in equivalents_numeric_columns:
  df1[i] = df1[i].apply(change_caracter)

In [37]:
def padronize_comma(column):
  column = column.apply(
        lambda value: value[:-3] + "." + value[-2:] if len(value) >= 3 else value
    )
  column = column.astype(float)

for i in equivalents_numeric_columns:
  padronize_comma(df1[i])

In [38]:
df1

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio
0,1,2023-09-03,META,"258,44","909,63","1380,05","595,86","61,54","8,846,040,350","3,57",1:1
1,2,2024-08-28,NVDA,"678,96","725,03","185,51","295,83","843,78",7346935699,"2,85",3:2
2,3,2022-05-10,INTU,"1348,94","215,00","831,86","874,05","336,19","8,027,589,419","2,28",2:1
3,4,2017-12-26,NVDA,"186,39","1111,85","508,65","511,49","952,63",1939.5M,"4,51",3:2
4,5,2022-01-20,WORK,"1425,94","692,20","340,75","378,58","1493,44",8984007687,"1,93",NaN
...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,2019-07-27,TSLA,"269,17","144,45","418,85","588,19","893,89",4184.54M,"4,71",3:2
199996,199997,2021-09-13,VMW,"315,78","262,78","89,13","172,09","659,42","5,782,458,138","1,04",3:2
199997,199998,2016-08-19,AAPL,"327,66","897,77","1402,05","140,90","904,11",3104296586,"3,21",NaN
199998,199999,2020-06-24,DOCU,"141,99","739,41","285,76","529,92","1201,64",45454067,"1,51",NaN


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [40]:
from datetime import timedelta

def get_operacao_por_data_ticker(ticker, data):
    t = yf.Ticker(ticker)

    data_dt = datetime.strptime(data, "%Y-%m-%d")
    data_fim = data_dt + timedelta(days=1)

    hist = t.history(start=data_dt.strftime("%Y-%m-%d"), end=data_fim.strftime("%Y-%m-%d"))

    if not hist.empty:

        return hist.iloc[0][['Open', 'High', 'Low', 'Close', 'Volume']]
    else:
        return None


valores = get_operacao_por_data_ticker('NVDA', '2024-08-28')
print(valores)


t = yf.Ticker('NVDA')
print(t.history(period="5d"))


Open      1.280884e+02
High      1.282984e+02
Low       1.226098e+02
Close     1.255791e+02
Volume    4.481011e+08
Name: 2024-08-28 00:00:00-04:00, dtype: float64
                                 Open        High         Low       Close  \
Date                                                                        
2025-05-16 00:00:00-04:00  136.220001  136.350006  133.460007  135.399994   
2025-05-19 00:00:00-04:00  132.389999  135.869995  132.389999  135.570007   
2025-05-20 00:00:00-04:00  134.289993  134.580002  132.619995  134.380005   
2025-05-21 00:00:00-04:00  133.059998  137.399994  130.589996  131.800003   
2025-05-22 00:00:00-04:00  132.229996  134.250000  131.550003  132.830002   

                              Volume  Dividends  Stock Splits  
Date                                                           
2025-05-16 00:00:00-04:00  226542500        0.0           0.0  
2025-05-19 00:00:00-04:00  193154600        0.0           0.0  
2025-05-20 00:00:00-04:00  161514200     

In [42]:
get_non_numeric_characters(df1["Volume"].apply( lambda x: x.strip()))

Volume
{',', 'i', 'b', 'n', 'B', 'K', '.', 'o', ' ', 'M', 'l'}


In [44]:
df1["Volume"]

for index, string in df1["Volume"].items():
  for char in string:
    if char.lower() in "bilion":
      new_string = ""
      for c in string:
        if c.lower() not in "bilion":
          new_string += c
      df1.loc[index,"Volume"] = new_string



In [53]:
def remove_letter(value):
  value = value.replace("M", "")
  value = value.replace("K", "")
  if value.count(",") > 1:
    value = value.replace(",", "")
  else:
    value = value.replace(",", ".")

  return float(value)

def transforming_float(value):
  value = value.strip()
  if "m" in value.lower():

    if remove_letter(value) >= 10000:
      return int(remove_letter(value) * 10**3)

    return int(remove_letter(value) * 10**6)

  return int(remove_letter(value))




df1["Volume"] = df1["Volume"].apply(transforming_float)


AttributeError: 'float' object has no attribute 'strip'

In [46]:
max_volume_index = df1['Volume'].idxmin()
minvol = df1["Volume"].min()
print(minvol)
print(max_volume_index)
print(df1.iloc[175058])

1012863.0
175058
ID                          175059
Date           2018-01-11 00:00:00
Ticker                        PYPL
Open                       1167,71
High                        169,68
Low                         497,93
Close                       590,70
Adj Close                   741,57
Volume                   1012863.0
Dividend                      4,37
Split Ratio                    2:1
Name: 175058, dtype: object


In [51]:
get_non_numeric_characters(df1["Split Ratio"].apply( lambda x: str(x)))

Split Ratio
{'Y', 'T', 'W', 'n', 'a', 'L', 'Z', 'G', 'H', 'o', 'M', 'V', 'F', 'U', 'e', ':', 'R', 'Q', 'X', 'B', 'O', 'J', 'D', 'K', 'I', 'P', 'E', 'A', 'N', 'C', 'S'}


In [108]:
df1

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio
0,1,2023-09-03,meta,258.44,909.63,1380.05,"595,86",61.54,"8,846,040,350",3.57,1:1
1,2,"Aug 28, 2024",NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2
2,3,"May 10, 2022",INTU,1348.94,215.00,831.86,874.05,"$336,19","8,027,589,419",2.28,2:1
3,4,2017-12-26,NVDA,$186.39,1111.85,508.65,511.49,$952.63,1939.5M,4.51,3:2
4,5,"Jan 20, 2022",WORK,1425.94,692.20,340.75,$378.58,1493.44,8984007687,1.93,nan
...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,"Jul 27, 2019",TSLA,"269,17",144.45,"418,85",588.19,$893.89,4184.54M,"4,71",3:2
199996,199997,"Sep 13, 2021",VMW,315.78,262.78,"89,13","$172,09",659.42,"5,782,458,138",1.04,3:2
199997,199998,2016-08-19,aapl,327.66,897.77,"1402,05",140.90,904.11,3104296586,$3.21,nan
199998,199999,2020-06-24,DJCU,141.99,739.41,285.76,$529.92,1201.64,45454067,1.51,nan


In [74]:
maxlen = 0
valor_max = ""
for _, valor in df1["Split Ratio"].items():
    valor_str = str(valor)
    if len(valor_str) >= maxlen:
        print(valor_str)
        maxlen = len(valor_str)
        valor_max = valor_str

print(maxlen)
print(valor_max)


1:1
3:2
2:1
3:2
nan
3:2
nan
2:1
2:1
2:1
nan
nan
3:2
1:1
1:1
1:1
1:1
NONE
NONE
NONE
none
none
none
NONE
NONE
NoCe
NONE
NONE
NONE
nonA
NONE
NLne
none
NONE
NONE
NONE
NONE
NONE
NONE
NONE
NoRe
Aone
NONE
none
NonI
NONE
none
none
NONE
NLne
none
NONE
none
NONE
NONE
none
NONE
none
NBne
NonY
none
none
NONE
NonX
NonX
NoXe
NONE
NONE
NBne
NONE
NONE
none
NONE
NYne
Done
NONE
NONE
NONE
none
NONE
NONE
NONE
NONE
NHne
NOME
none
NONE
NONE
none
none
NONE
none
NONE
NONE
none
none
none
none
NONE
none
NonP
NONE
none
none
none
NMne
NONE
none
none
noQe
NONE
nWne
none
NONE
NONE
none
NONE
NONE
NonM
none
none
none
NONE
NONE
NONE
none
none
NONE
NONE
none
NONE
none
NWne
none
none
Vone
none
Yone
NONG
NONE
nonC
NONE
none
none
NONE
none
NNne
NONE
NONE
NAne
NONE
NONE
NONE
none
none
NONE
NONE
NPne
none
NONE
none
none
Uone
NONE
none
NONE
NonI
NONE
none
NoZe
NONE
none
NSne
none
NONE
none
NONE
NoWe
NOSE
none
NONE
NONE
none
NONE
NoJe
NoGe
NONE
NONE
none
none
none
NONE
NONE
NONE
NonF
none
none
NONE
NONE
NONE
none
NONE
none
NQ

In [101]:
def valid_split(valor):
  return valor[0].isdigit() and valor[1] != ":" and valor[2].isdigit()

for i, valor in df1["Split Ratio"].items():
    valor = str(valor)
    if len(valor_str) == 3:
        if valid_split(valor):
            print(valor)
            df1.loc[i, "Split Ratio"] = valor[0] + ":" + valor[2]
            print(df1.loc[i, "Split Ratio"])


In [107]:
def right_split(val):
    val = str(val)
    return len(val) == 3 and val[0].isdigit() and val[1] == ":" and val[2].isdigit()

poss = []

for _, val in df1["Split Ratio"].items():
    if pd.isna(val):
        continue

    if right_split(val) and val not in poss:
        poss.append(val)

print("Split Ratios válidos encontrados:", poss)

# A:1 


Split Ratios válidos encontrados: ['1:1', '3:2', '2:1']


In [4]:
def ajust_parcial_splits(val):

  if val[0] == "2":
    return val[:-1] + "1"
  if val[0] == "3":
    return val[:-1] + "2"
  if val[2]


test = "2:a"
print(ajust_parcial_splits(test))


2:1


In [111]:
c = 0
d = 0
tot = 0
for i, v in df1["Split Ratio"].items():
  tot += 1
  if right_split(v) and v[2]== "1":
    if v[0] == "1":
      c+=1
    elif v[0] == "2":
      d+=1
print(c/tot)
print(d/tot)

0.240595
0.2426
